In [ ]:
pip install confluent-kafka pandas faker

In [ ]:
from confluent_kafka import Producer, Consumer
import pandas as pd
import random
import faker

fake =faker.Faker()

def generate_order_data(num_orders):
    order_ids = []
    customer_names = []
    product_ids = []
    quantities = []
    prices = []
    order_dates = []
    order_delivery_address=[]

    for _ in range(num_orders):
        order_ids.append(fake.uuid4())
        customer_names.append(fake.name())
        product_ids.append(random.randint(1, 20))
        quantities.append(random.randint(1, 50))
        prices.append(round(random.uniform(10, 1000), 2))
        order_dates.append(fake.date_this_year())
        order_delivery_address.append(fake.address())

    data = {
        'Order ID': order_ids,
        'Customer Name': customer_names,
        'Product ID': product_ids,
        'Quantity': quantities,
        'Price': prices,
        'Order Date': order_dates,
        'Order Delivery Address': order_delivery_address
    }

    return pd.DataFrame(data)
    
def read_config():
  # reads the client configuration from client.properties
  # and returns it as a key-value map
  config = {}
  with open("client.properties") as fh:
    for line in fh:
      line = line.strip()
      if len(line) != 0 and line[0] != "#":
        parameter, value = line.strip().split('=', 1)
        config[parameter] = value.strip()
  return config

def produce(topic, config,df):
    producer = Producer(config)
    for _, row in df.iterrows():
        message = row.to_json()
        producer.produce('orders', value=message)
        producer.flush()
    
def consume(topic, config):
    config["group.id"] = "python-group-1"
    config["auto.offset.reset"] = "earliest"

  # creates a new consumer instance
    consumer = Consumer(config)

  # subscribes to the specified topic
    consumer.subscribe([topic])
    consumer.close()

def main():
  num_of_orders =50
  config = read_config()
  topic = "orders"

  df = generate_order_data(num_of_orders)
  produce(topic, config,df)
  consume(topic, config)
    

main()